In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import itertools
sns.set(style="whitegrid", font_scale=1.5)
import sys
# sys.path.append("../")

import benchmarks
from qm import QueryManager
import argparse
import time


import skopt
from opt_skopt import optimize

In [5]:
dataset = 'adult'
workload = 64
marginal = 3
epsilon=0.25
max_iter = 10

In [6]:
data, workloads = benchmarks.randomKway(dataset, workload, marginal)
N = data.df.shape[0]
stime = time.time()
query_manager = QueryManager(data.domain, workloads)

In [7]:
opt =  optimize(data=data,
                     query_manager=query_manager,
                     epsilon=epsilon,
                     samples=200,
                     max_iter=max_iter,
                     timeout=30,
                     show_plot=False)

100%|██████████| 9/9 [00:24<00:00,  2.68s/it]


In [13]:
fig = plt.figure()   
ax = skopt.plots.plot_convergence(opt)
plt.savefig('out.png')

In [16]:
fig = plt.figure()   
ax = skopt.plots.plot_evaluations(opt)
plt.draw()

In [20]:
fig = plt.figure(figsize=plt.figaspect(0.7)) 
skopt.plots.plot_objective(opt)
plt.savefig('out.png')

In [28]:
from mpl_toolkits.mplot3d import Axes3D

ins = np.array(opt.x_iters)
outs = opt.func_vals
eps0 = ins[:,0]
noise = ins[:,1]
error = outs

In [29]:
noise

array([0.75426375, 1.7751807 , 1.19639549, 2.19114798, 0.86667344,
       1.27346922, 0.85603541, 0.75      , 3.        , 0.75      ])

In [31]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Z_map = {}

for i, (e,n) in enumerate(zip(eps0, noise)):
    if e not in Z_map:
        Z_map[e] = {}
    Z_map[e][n] = error[i]

/Users/adamhuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


{0.001708905540832146: {0.75: 0.7207731051144507},
 0.006682686356950824: {0.7542637463621724: 0.7207731051144507},
 0.007112808143062447: {0.856035410115411: 0.5897178657712624},
 0.008219499061269557: {0.8666734430979179: 0.7207731051144507},
 0.009446649971241092: {1.1963954941935695: 0.7207731051144507},
 0.011548483163805655: {1.273469215488333: 0.39560606725253533},
 0.011689679154494544: {1.7751806962034267: 0.306008592588305},
 0.011952735455184764: {2.1911479848418676: 0.3070525193044853},
 0.01196776080835247: {3.0: 0.535309445966581}}

In [39]:
Z_map[0.001708905540832146][0.75]

0.7207731051144507

In [40]:
eps0 = np.unique(eps0)
noise = np.unique(noise)
X, Y = np.meshgrid(eps0, noise)
Z = np.array([ [ Z_map[e][n] for e in eps0]  for n in noise])
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

plt.show()

KeyError: 0.75

In [ ]:


fig, ax = plt.subplots()

x1_values = np.linspace(0.001, epsilon/20, 100)
x2_values = np.linspace(0.75, 3, 100)
x_ax, y_ax = np.meshgrid(x1_values, x2_values)
vals = np.c_[x_ax.ravel(), y_ax.ravel()]
fx = np.reshape([branin(val) for val in vals], (100, 100))

cm = ax.pcolormesh(x_ax, y_ax, fx,
                   norm=LogNorm(vmin=fx.min(),
                                vmax=fx.max()))

minima = np.array([[-np.pi, 12.275], [+np.pi, 2.275], [9.42478, 2.475]])
ax.plot(minima[:, 0], minima[:, 1], "r.", markersize=14,
        lw=0, label="Minima")

cb = fig.colorbar(cm)
cb.set_label("f(x)")

ax.legend(loc="best", numpoints=1)

ax.set_xlabel("$X_0$")
ax.set_xlim([-5, 10])
ax.set_ylabel("$X_1$")
ax.set_ylim([0, 15])


